# Catch-Up AI Training with a Q-table

Import all the necessary python libraries

In [1]:
import numpy as np
import itertools

Define all the function that will be use to simulate a game of Catch-Up.

In [2]:
def gameOver(state):
    '''
    gameOver() returns True if there are no more pieces to be chosen.
    '''
    piecesLeft = (sum(state[:n]))
    return not piecesLeft

def posAction(state):
    '''
    posAction(state) will given a state returns a list of all the pieces
    that could be chosen.
    '''
    return [x+1 for x in range(n) if state[x] == 1]

def nextState(state, action):
    '''
    nextState(state, action) transforms one state of the games into
    the next one given the taken action.
    '''
    state = list(state)
    state[action - 1] = 0
    state[n] += action
    if state[n] > state[n+1]:
        state = switchPlayer(state)
    return tuple(state)

def switchPlayer(state):
    '''
    switchPlayer(state) transforms the state as to make sure that the 
    player with the lower score is on play.
    '''
    state = list(state)
    state[n], state[n+1] = state[n+1], state[n]
    return tuple(state)

def determineReward(state):
    '''
    determineReward(state) returns a reward if the player wins or loses
    a game of catch-up.
    '''
    if gameOver(state) and state[n] > state[n+1]:
        return 1
    elif gameOver(state) and state[n] < state[n+1]:
        return -1
    else:
        return 0
    
def buildStateSpace(state, first = True):
    '''
    buildStateSpace(state, first = True) will return a generator that
    runs over all the possible game states. Take in mind that 
    duplicates will be included in this iteration.
    '''
    if first: yield state
    posAct = posAction(state)
    if len(posAct) > 0:
        for action in posAct:
            newState = nextState(state, action)
            yield newState
            for x in buildStateSpace(newState, False):
                yield x
    else:
        yield state

In [153]:
class qTable(object):
    '''
    qTable() is a class object that contains the Q table and
    is able to do operations with this table.
    '''
    
    def __init__(self, n, learningRate, discountRate):
        '''
        Initializes the qTable object.
        '''
        self.lr = learningRate
        self.dr = discountRate        
        self.table = {}
        self.trans = {}
        combTable = []
            
        state = tuple([1 if i < n else 0 for i in range(n+2)])
        for piece in buildStateSpace(state, True):
            self.table[piece] = [0 for _ in range(n)]
            # (np.random.random()-0.5)*2
            
    def update(self, state, newState, action, reward, lr, dr):
        '''
        update(self, state, newState, action, reward, lr, dr) updates
        the Q table given the information gained from the reward, next and
        current state. Both learning and discount rate of the model are
        required inputs as well.
        '''
        self.table[state][action-1] = (1 - lr) * self.table[state][action-1] + \
        lr * (reward + dr * max(self.table[newState]))
        
    def translate(self):
        '''
        translate() translates the values in the Q table into a table
        that gives the suggest move for each state.
        '''
        for x in self.table.keys():
            posAct = posAction(x)
            posQ = self.table[x].copy()
            action = 0
            while len(posAct) > 0:
                action = posQ.index(max(posQ)) + 1
                if action in posAct:
                    break
                else:
                    posQ[action - 1] = min(posQ) - 1
            self.trans[x] = action

In [131]:
def botRandom(state):
    '''
    Bot chooses any move at random.
    '''
    posAct = posAction(state)
    diff = state[n+1] - state[n]
    combAct = []
    while (diff > 0 or len(combAct) == 0) and not len(posAct) == 0:
        action = np.random.choice(posAct)
        combAct.append(action)
        posAct.remove(action)
        diff = state[n+1] - state[n] - sum(combAct)
        
    return combAct

def botMaxScore(state):
    '''
    Bot maximizes its scores on every turn, 
    extending their leads by as much as possible.
    '''
    posAct = posAction(state)
    diff = state[n+1] - state[n]
    combAct = []
    
    # Add the biggest possible pieces that don't trigger
    # a change in player.
    while (min(posAct) < diff or len(combAct) == 0) and not len(posAct) == 0:
        action = max([x for x in posAct if x < diff])
        combAct.append(action)
        posAct.remove(action)
        diff = state[n+1] - state[n] - sum(combAct)
    
    # Add the biggest possible piece to create the biggest
    # extension.
    combAct.append(max(posAct))

    return combAct

def botMinScore(state):
    '''
    Bot minimizes its scores on every turn
    keeping their scores as close as possible.
    '''
    posAct = posAction(state)
    diff = state[n+1] - state[n]
    combAct = []
    
    # Add the biggest possible pieces that don't trigger
    # a change in player.
    while (min(posAct) < diff or len(combAct) == 0) and not len(posAct) == 0:
        action = max([x for x in posAct if x < diff])
        combAct.append(action)
        posAct.remove(action)
        diff = state[n+1] - state[n] - sum(combAct)
    
    # Add the smallest possible piece to create the smallest
    # extensions but not equal.
    combAct.append(min(posAct))
    
    return combAct

def botUseMostNums(state):
    '''
    Bot uses as many numbers as possible, reducing
    the numbers available for the opponent.   
    '''
    posAct = posAction(state)
    diff = state[n+1] - state[n]
    combAct = []
    
    while (min(posAct) < diff or len(combAct) == 0) and not len(posAct) == 0:
        action = min(posAct)
        combAct.append(action)
        posAct.remove(action)
        diff = state[n+1] - state[n] - sum(combAct)
        
    # Add the biggest possible piece to create the biggest
    # extension.
    combAct.append(max(posAct))

    return combAct

In [132]:
def playerAI(state, qTable):
    '''
    The AI will look into the Q table for the action with
    the highest future reward and pick that action. This
    function recurses until the AI's turn lapses.
    '''
    posAct = posAction(state)
    diff = state[n+1] - state[n]
    
    posQ = qTable.table[state].copy()
    for _ in range(len(posQ)):
        action = posQ.index(max(posQ)) + 1
        if action in posAct:
            yield action
            if diff - action > 0:
                newState = nextState(state, action)
                yield from playerAI(newState, qTable)
            break
        else:
            posQ[action - 1] = min(posQ) - 1

In [154]:
learningRate = 0.1
discountRate = 0.9
n = 4
reward = 10

# initialize the Q table
qTab = qTable(n, learningRate, discountRate)

for _ in range(1000):
    # assemble starting state
    state = tuple([1 if i < n else 0 for i in range(n+2)])
    # while the game is not finished
    while not gameOver(state):
        # Player One (Bot)
        actions = botRandom(state)

        for action in actions:
            state = nextState(state, action)
        
        if not gameOver(state):
        # Player Two (AI)
            actions = list(playerAI(state, qTab))
        
            for action in actions:
                # find the next state corresponding to the action selected
                newState = nextState(state, action)
                if state[n] + action > state[n+1]:
                    lrFactor = -1
                else:
                    lrFactor = 1
                # get reward factor
                rewardFactor = determineReward(newState)
                # update Q table
                qTab.update(state, newState, action, 
                                 rewardFactor * reward, 
                                 learningRate * lrFactor, discountRate)
                # got to new state
                state = newState      
            
    #print("Episode ", _, " completed.")
    
qTab.translate()
qTab.trans

{(1, 1, 1, 1, 0, 0): 1,
 (0, 1, 1, 1, 0, 1): 2,
 (0, 0, 1, 1, 1, 2): 4,
 (0, 0, 0, 1, 2, 4): 4,
 (0, 0, 0, 0, 4, 6): 0,
 (0, 0, 1, 0, 2, 5): 3,
 (0, 0, 0, 0, 5, 5): 0,
 (0, 1, 0, 1, 1, 3): 2,
 (0, 0, 0, 1, 3, 3): 4,
 (0, 0, 0, 0, 3, 7): 0,
 (0, 1, 0, 0, 3, 5): 2,
 (0, 1, 1, 0, 1, 4): 2,
 (0, 0, 1, 0, 3, 4): 3,
 (0, 1, 0, 0, 4, 4): 2,
 (1, 0, 1, 1, 0, 2): 1,
 (1, 0, 0, 1, 2, 3): 1,
 (1, 0, 0, 0, 3, 6): 1,
 (1, 0, 1, 0, 2, 4): 1,
 (1, 0, 0, 0, 4, 5): 1,
 (1, 1, 0, 1, 0, 3): 1,
 (1, 1, 0, 0, 3, 4): 1,
 (1, 1, 1, 0, 0, 4): 1}